### Gorilla RAFT setup section

This cell is just a section header comment that marks the start of the Gorilla RAFT setup steps used to generate synthetic training data.
This will execute the `setup_raft.sh` helper script, which prepares the Gorilla RAFT environment (e.g., cloning the RAFT repo, installing its dependencies, and setting up any required directories or configuration files).

In [ ]:
! ./setup_raft.sh

### Install Python dependencies for the workshop

This cell installs all Python packages required for the data generation pipeline, including environment management (`dotenv`), data handling (`pandas`, `datasets`), model tooling (`openai`, `transformers`, LangChain libraries), Azure identity helpers(not used dependency of gorilla raft), logging (`coloredlogs`), and any RAFT-related utilities.

In [ ]:
! pip install dotenv pandas mdc openai datasets transformers PyPDF2 langchain_experimental langchain_openai azure-identity coloredlogs

### Load configuration and define dataset settings

This cell sets up the basic experiment parameters:

- Loads environment variables (dataset name, paths, models).
- Defines how much data goes to training vs. validation.(splits)
- Sets a QA limit so RAFT doesn’t generate too many questions per document.
- Prints a summary so you can confirm all settings before generating data.

In [ ]:
import os
from math import ceil
from dotenv import load_dotenv

# ---------------------------------------------------------------------
# LOAD CONFIGURATION
# ---------------------------------------------------------------------
load_dotenv("config.env")

# ---------------------------------------------------------------------
# DATASET CONFIGURATION
# ---------------------------------------------------------------------
ds_name = os.getenv("DATASET_NAME")
ds_file = os.getenv("DATASET_FILE")
os.environ["DATAFILE_PATH"] = f"sample_data/{ds_name}/{ds_file}"
os.environ["FORMAT"] = "chat"

# Define dataset output paths
ds_path = f"dataset/{ds_name}"
os.environ["DATASET_OUTPUT_PATH"] = ds_path

# ---------------------------------------------------------------------
# TRAINING PARAMETERS
# ---------------------------------------------------------------------
finetuning_train_split = 0.8
finetuning_valid_split = 0.1
finetuning_threshold = 65
raft_questions = 2
qa_threshold = ceil(finetuning_threshold / finetuning_train_split)

# ---------------------------------------------------------------------
# PRINT CONFIGURATION SUMMARY
# ---------------------------------------------------------------------
print(
    f"""
═══════════════════════════════════════════════════════════════════════
    RAFT Synthetic Dataset Generation - Configuration Overview
═══════════════════════════════════════════════════════════════════════

MODEL & ROUTER
──────────────────────────────────────────────
 Multi-Model Router URL  : {os.getenv('OPENAI_BASE_URL')}
 Teacher Model           : {os.getenv('TEACHER_MODEL_ID')}
 Embedding Model         : {os.getenv('EMBEDDING_MODEL_ID')}

DATASET SETUP
──────────────────────────────────────────────
 Dataset Name            : {ds_name}
 Dataset File            : {ds_file}
 Training Document Path  : {os.getenv('DATAFILE_PATH')}
 Output Dataset Path     : {ds_path}

TRAINING PARAMETERS
──────────────────────────────────────────────
 Finetuning Train Split  : {finetuning_train_split}
 Finetuning Valid Split  : {finetuning_valid_split}
 Finetuning Threshold    : {finetuning_threshold}
 QA Threshold (derived)  : {qa_threshold}
 Questions per Chunk     : {raft_questions}

═══════════════════════════════════════════════════════════════════════
"""
)

### Run Gorilla RAFT to generate synthetic QA data

This cell invokes the Gorilla RAFT CLI (`raft.py`) to:
- Ingest the source document(s) from `$DATAFILE_PATH` (PDF),
- Chunk the document into 512-token segments,
- Generate 2 questions per chunk, including **distractors** (plausible but incorrect answer choices),
- Use the specified embedding and completion models.

The output is written to `$DATASET_OUTPUT_PATH` as an **Arrow file** (a columnar data format that’s efficient for large datasets) that will be used in later steps.

>Why include distractors?
>Distractors are intentionally wrong but believable answers. They help the model learn to distinguish correct information from similar false options improving >comprehension and reasoning rather than simple memorization.

In [ ]:
!python3 .gorilla/raft/raft.py \
    --datapath "$DATAFILE_PATH" \
    --output "$DATASET_OUTPUT_PATH" \
    --doctype pdf \
    --chunk_size 512 \
    --questions 2 \
    --distractors 3 \
    --embedding_model "$EMBEDDING_MODEL_ID" \
    --completion_model "$TEACHER_MODEL_ID"

### Define intermediate dataset file paths and summarize them

This cell:
- Records the main RAFT Arrow file path and exports it as `RAFT_ARROW_FILE`,
- Builds paths for the intermediate HF-style JSONL files (full, train, valid, eval),
- Builds paths for the final finetuning JSONL files (train and valid),
- Stores all of these paths in environment variables.(passed to raft cli)
- Prints a summary of the intermediate and final dataset file locations for quick reference.

#### HF style JSONL files
>A Hugging Face (HF) style JSONL file stores one training example per line in plain text.
>Each line is a small JSON object containing fields like "instruction", "input", and "output".
>This makes it easy to preview, edit, and load datasets with the Hugging Face datasets library or similar tools.

In [ ]:
raft_arrow_file = f"{ds_path}/data-00000-of-00001.arrow"
os.environ["RAFT_ARROW_FILE"] = raft_arrow_file

dataset_path_hf = f"{ds_path}-files/{ds_name}-hf.full.jsonl"
os.environ["DATASET_PATH_HF"] = dataset_path_hf

dataset_path_hf_train = f"{ds_path}-files/{ds_name}-hf.train.jsonl"
os.environ["DATASET_PATH_HF_TRAIN"] = dataset_path_hf_train
dataset_path_hf_valid = f"{ds_path}-files/{ds_name}-hf.valid.jsonl"
os.environ["DATASET_PATH_HF_VALID"] = dataset_path_hf_valid
dataset_path_hf_eval  = f"{ds_path}-files/{ds_name}-hf.eval.jsonl"

dataset_path_ft_train = f"{ds_path}-files/{ds_name}-ft.train.jsonl"
os.environ["DATASET_PATH_FT_TRAIN"] = dataset_path_ft_train
dataset_path_ft_valid = f"{ds_path}-files/{ds_name}-ft.valid.jsonl"
os.environ["DATASET_PATH_FT_VALID"] = dataset_path_ft_valid

print(
    f"""
Intermediate Dataset Files
--------------------------
RAFT arrow file        : {raft_arrow_file}

HF JSONL (synthetic data)
  Full dataset         : {dataset_path_hf}
  Train split          : {dataset_path_hf_train}
  Valid split          : {dataset_path_hf_valid}
  Eval split           : {dataset_path_hf_eval}

Finetuning JSONL (final RAFT-style)
  Train split          : {dataset_path_ft_train}
  Valid split          : {dataset_path_ft_valid}
"""
)


### Convert RAFT Arrow dataset to HF JSONL format

This cell uses the RAFT `format.py` helper to convert the Arrow file (`$RAFT_ARROW_FILE`) into a Hugging Face–style JSONL file at `$DATASET_PATH_HF`.  
The `hf` output format makes it easier to inspect and manipulate with tools like `pandas` and `datasets`.


In [ ]:
! python .gorilla/raft/format.py \
    --input "$RAFT_ARROW_FILE" \
    --output "$DATASET_PATH_HF" \
    --output-format hf

### Load and preview the full HF JSONL dataset

This cell imports `pandas`, reads the full synthetic dataset from `$DATASET_PATH_HF` into a DataFrame, and shows the first five rows.  
It provides a quick sanity check that RAFT successfully generated instructions, questions, answers, and contexts.


In [ ]:
# Preview the synthetic dataset generated in the HF JSONL stage
import pandas as pd

hf_full_df = pd.read_json(dataset_path_hf, lines=True)
hf_full_df.head(5)

### Render a formatted sample from the HF dataset

This cell selects an example from `hf_full_df` and:
- Cleans up `<DOCUMENT>` and similar tags for Markdown display,
- Builds a formatted view showing the **oracle context**, **question**, and **chain-of-thought answer**,
- Displays it neatly with `IPython.display.Markdown`.

The **oracle context** shows the exact text the model used to generate the question.  
The **question** tests the model’s understanding of that context.  
The **chain-of-thought answer** reveals the reasoning path, not just the final answer, helping verify that the generated data teaches logical reasoning instead of recall from memory.

In [ ]:
# Display a random sample from the HF dataset to inspect structure and content
from IPython.display import display, Markdown
from random import randint

sample_idx = 2
sample = hf_full_df.iloc[sample_idx]

instruction_md = sample.instruction.replace("<DOCUMENT>", "`<DOCUMENT>`").replace("</DOCUMENT>", "`</DOCUMENT>`")
oracle_context_md = sample.oracle_context.replace("<DOCUMENT>", "`<DOCUMENT>`").replace("</DOCUMENT>", "`</DOCUMENT>`")
sample_answer_md = sample.cot_answer.replace("<ANSWER>", "`<ANSWER>`").replace("##begin_quote##", "`##begin_quote##`").replace("##end_quote##", "`##end_quote##`")

display(Markdown(f"""
## Oracle Context
{oracle_context_md}

## Question
{sample.question}

## CoT Answer
{sample_answer_md}
"""))

### Split HF dataset into train/valid/eval JSONL files

This cell:
- Computes the index cut points for train and validation splits using the configured ratios,
- Logs how many samples land in each split and where they will be saved,
- Uses `numpy.split` to partition `hf_full_df` into train, validation, and eval DataFrames,
- Writes each split out as line-delimited JSONL files at the configured paths.

These HF JSONL splits are the basis for later finetuning and evaluation datasets.


In [ ]:
# Split the HF JSONL dataset into train/valid/eval splits and write them to disk
import numpy as np

samples_count = len(hf_full_df)
train_cut = int(finetuning_train_split * samples_count)
valid_cut = int((finetuning_train_split + finetuning_valid_split) * samples_count)
splits = [train_cut, valid_cut]

print(
    f"""
Splitting HF dataset
--------------------
Total samples : {samples_count}
Train split   : 0 -> {train_cut}        -> {dataset_path_hf_train}
Valid split   : {train_cut} -> {valid_cut} -> {dataset_path_hf_valid}
Eval split    : {valid_cut} -> {samples_count} -> {dataset_path_hf_eval}
"""
)

hf_train_df, hf_valid_df, hf_eval_df = np.split(hf_full_df, splits)

hf_train_df.to_json(dataset_path_hf_train, orient="records", lines=True)
hf_valid_df.to_json(dataset_path_hf_valid, orient="records", lines=True)
hf_eval_df.to_json(dataset_path_hf_eval, orient="records", lines=True)

### Convert HF train split into RAFT-style finetuning dataset

This cell calls `format.py` again to:
- Take the HF train JSONL (`$DATASET_PATH_HF_TRAIN`) as input,
- Produce a RAFT-style finetuning JSONL (`$DATASET_PATH_FT_TRAIN`),
- Map the question/answer fields into a standard schema where `text` is the prompt and `ground_truth` is the completion.

The resulting file is directly usable as the finetuning dataset for the model.

In [ ]:
!python .gorilla/raft/format.py \
    --input "$DATASET_PATH_HF_TRAIN" \
    --input-type jsonl \
    --output "$DATASET_PATH_FT_TRAIN" \
    --output-format "$FORMAT" \
    --output-completion-prompt-column text \
    --output-completion-completion-column ground_truth

### Convert HF validation split into RAFT-style finetuning dataset

This cell performs the same RAFT `format.py` transformation as above, but for the validation split:
- Input: `$DATASET_PATH_HF_VALID` (HF JSONL),
- Output: `$DATASET_PATH_FT_VALID` (RAFT-style JSONL),
- Uses the same `text` / `ground_truth` mapping.

This RAFT-style validation set will be used to gauge finetuning performance.

In [ ]:
! python .gorilla/raft/format.py \
    --input "$DATASET_PATH_HF_VALID" \
    --input-type jsonl \
    --output "$DATASET_PATH_FT_VALID" \
    --output-format "$FORMAT" \
    --output-completion-prompt-column text \
    --output-completion-completion-column ground_truth

### Inspect finetuning and evaluation datasets

This cell:
- Loads the RAFT-style finetuning validation dataset from `dataset_path_ft_valid` and shows its first two rows,
- Loads the HF eval JSONL (`dataset_path_hf_eval`) and shows its first two rows.

It provides a final verification that both the finetuning and evaluation splits are correctly formatted and ready for the next notebooks.

In [ ]:
# Inspect finetuning and eval splits
dataset_path_ft_valid_df = pd.read_json(dataset_path_ft_valid, lines=True)
dataset_path_ft_valid_df.head(2)

pd.read_json(dataset_path_hf_eval, lines=True).head(2)